In [1]:
import numpy as np # NumPy
import matplotlib.pylab as plt # Matplotlib plots
import matplotlib.patches as mpatches
import matplotlib.colors as colors
import mpl_scatter_density     # density scatter plots
import pandas as pd # Pandas
import uproot
import pickle
import logging

import os # read directories etc.
from scipy.signal import find_peaks, gaussian
from scipy.optimize import curve_fit
from scipy.stats import gaussian_kde
import pylandau

import time
from timeout_decorator import timeout

from LoadBatch import *
from SensorClasses import *

In [2]:
pd.set_option('display.max_columns', None)
# logging.basicConfig(level=logging.WARNING, format='[%(levelname)s] - %(message)s')
logging.basicConfig(level=logging.INFO, format='[%(levelname)s] - \t %(message)s')
# plt.rcParams['text.usetex'] = True   ### LateX in matplotlib
# !ls ../Data_TestBeam/2023_May/

In [3]:
# plt.style.use(hep.atlas.style.ATLAS)

### Bins options, sensors and runs import

In [4]:
PIXEL_SIZE = 0.0185 #mm

### choose the bins so that they match the MIMOSA pixels (which are just the coordinates)
large_bins = (np.arange(0, 900,1),
              np.arange(0, 600,1))

bins1 = (np.arange(450, 700, 1),
        np.arange(100, 500, 1))

bins2 = (np.arange(350, 700, 1),
              np.arange(150, 500, 1))

bins3 = (np.arange(300, 800, 1),
              np.arange(0, 450, 1))

bins4 = (np.arange(500, 800, 1),
              np.arange(100, 500, 1))

# small_bins = (np.arange(500, 650, 1),
#               np.arange(200, 450, 1))

### Load the dictionary of sensor names and runs
# dict_of_runs = read_pickle("dict_of_runs.pickle")
dict_of_batches = read_pickle("dict_of_batches.pickle")


# ### presentation path
pres_path = '/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Presentations/2024-02-15 for fast-silicon-pixel'

In [5]:
my_list = list(dict_of_batches.keys())
my_list.sort()
print(my_list)

[100, 101, 199, 201, 202, 203, 204, 205, 206, 301, 401, 402, 403, 407, 408, 409, 410, 411, 413, 414, 501, 502, 503, 504, 505, 601, 602, 603, 604, 605, 701, 702, 801, 802, 901, 902, 1001, 1002, 1101, 1102, 1201, 1202]


I kinda want a 'reversed' dictionary:
for each sensor_name: have a list of batches

In [6]:
def get_DUTs_from_dictionary(dictionary, oscilloscope):
    ### I only add the dut if there is a 'board' and voltage>0
    DUTs = []
    for i,ch in enumerate(('Ch2','Ch3','Ch4')):
        if (dictionary[(oscilloscope,ch)]['board'] != 'no_board') and (dictionary[(oscilloscope,ch)]['voltage'] != 0):
            DUTs.append(i+1)
    return DUTs

## Plot a single batch

In [7]:
### show all information about the batch
this_batch = 501
dir_path = f'/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/various plots/all batches/{this_batch}'

display(dict_of_batches[this_batch].__dict__)

info_dict = {}
### show all informations about each sensor
for S in ['S1','S2']:
    for ch, sensor in dict_of_batches[this_batch].S[S].channels.items():
#         print(f"{S}, {ch}:", sensor.__dict__)
        info_dict[(S,ch)] = sensor.__dict__
info_df = pd.DataFrame(info_dict)
# Export DataFrame to a CSV file
info_df.to_csv(os.path.join(dir_path,f'table_data_{this_batch}.csv'), index=True)

display(info_df)

{'batch_number': 501,
 'angle': 0.0,
 'runs': [9796, 9797, 9798, 9799, 9800, 9801, 9802, 9803, 9804, 9805],
 'tempA': -26.57,
 'tempB': -26.77,
 'S': {'S1': <SensorClasses.Oscilloscope at 0x7f14db4141c0>,
  'S2': <SensorClasses.Oscilloscope at 0x7f14db414460>}}

S1                                                  \
                     Ch1                      Ch2                    Ch3   
name                 MCP  JSI-PP4-IMEv2-W7-6.5E14  JSI-B7-IMEv3-W16-8E14   
board           no_board                  JSI PP4                 JSI B7   
dut_position         7.0                      1.0                    2.0   
fluence                0                 6.5E14 p               8.00E+14   
transimpedance        -1                     4700                   4700   
voltage             2800                     -200                   -200   

                                                S2                        \
                                     Ch4       Ch1                   Ch2   
name            JSI-B13-IMEv3-W16-2.5E15       MCP  JSI-B6-IMEv2-W7-1E14   
board                            JSI B13  no_board                JSI B6   
dut_position                           3       7.0                     4   
fluence                         2.50E+15         0                1E14 P   
transimpedance                      4700        -1                  4700   
voltage                             -400      2800                   -80   

                                    
                     Ch3       Ch4  
name             JSI-PP1            
board            JSI PP1  no_board  
dut_position           6       NaN  
fluence         1.50E+15         0  
transimpedance      4700        -1  
voltage             -400         0

In [8]:
print('S1:', get_DUTs_from_dictionary(info_dict,'S1'))
print('S2:', get_DUTs_from_dictionary(info_dict,'S2'))


S1: [1, 2, 3]
S2: [1, 2]


In [ ]:
SAVE = True

colormap = ['k','b','g','r']

threshold_charge = 4 #fC
MCP_resolution = 36.52

CFD_values = (20, 50,70)
axes_size = len(CFD_values)
window_limit = 20e3

print("Batch: ", this_batch)

if not os.path.exists(dir_path):
    os.mkdir(dir_path)

### save in a presentation folder
# dir_path = pres_path

df = {}  # dictionary containing the two dataframes of the two oscilloscopes
use_for_geometry_cut = 'time'
binning_method = 'rice'
these_bins = bins2    ### custom bins around the sensors
time_bins = 5000
for S in ['S1','S2']: #"S2" ### the two scopes
    print(S)
    df[S] = load_batch(this_batch,S)
    print(f'MCP: {dict_of_batches[this_batch].S[S].channels["Ch1"].voltage} V, angle: {dict_of_batches[this_batch].angle}°', 'temperature:%.2f°C'%dict_of_batches[this_batch].tempA)
    ### I PUT THE TRANSIMPEDANCE TO 4700 MANUALLY

    DUTs = get_DUTs_from_dictionary(info_dict,S)
    
    ### show full area d
    plot(df[S], "2D_Tracks", dict_of_batches[this_batch], S, bins=large_bins,
         n_DUT=DUTs, savefig=SAVE, savefig_details=f' {S} all tracks (no cut)', savefig_path=dir_path, fmt='png')  
    
    my_transimpedance = 4700

    ###[ ... if dut in DUTs else None for dut in [1,2,3]], it simply doesn't calculate the cuts for the channels with no dut
    geo_cuts = [geometry_mask(df[S], DUT_number=dut, bins=these_bins, bins_find_min='rice', use=use_for_geometry_cut)[0] if dut in DUTs else None for dut in [1,2,3]]
    central_sensor_area_cuts = [geometry_mask(df[S], DUT_number=dut, bins=these_bins, bins_find_min='rice', only_select='center', use=use_for_geometry_cut)[0] if dut in DUTs else None for dut in [1,2,3]]
    time_cut = [time_mask(df[S], dut, bins=time_bins, mask=geo_cuts[dut-1], plot=False, savefig=os.path.join(dir_path,f'time_plot_with_geo_cuts_{S}_{this_batch}_DUT{dut}.png'))[0] if dut in DUTs else None for dut in [1,2,3]]
    mins = [find_min_btw_peaks(df[S][f"pulseHeight_{dut}"], bins='rice', plot=False) if dut in DUTs else None for dut in [1,2,3]]
    pulseheight_cut = [df[S][f'pulseHeight_{dut}']>mins[dut-1] if dut in DUTs else None for dut in [1,2,3]]
    charge_cut = [df[S][f'charge_{dut}']>threshold_charge if dut in DUTs else None for dut in [1,2,3]]

    # charge_params = [charge_fit(df[S], dut=dut, mask=np.logical_and(time_cut[dut-1],geo_cuts[dut-1]), transimpedance=my_transimpedance, plot=True,
    #                             savefig=os.path.join(dir_path,f'charge_fit_{S}_{this_batch}_DUT{dut}.png'))[0] if dut in DUTs else None for dut in [1,2,3]]
    # time_resolution = [time_mask(df, dut, bins=time_bins, mask=np.logical_and(np.logical_and(geo_cuts[dut-1],pulseheight_cut),charge_cut), plot=True,
    #                              savefig=os.path.join(dir_path,f'time_plot_with_geo_cuts_{S}_{this_batch}_DUT{dut}.png'))[1]['parameters'][2] if dut in DUTs else None for dut in [1,2,3]]
    #     ### show sensor by doing time cut
#         plot(df[S], "2D_Tracks", dict_of_batches[this_batch], S, bins=these_bins, mask=time_cut,
#              n_DUT=DUTs, savefig=SAVE, savefig_details=f' {S} (w. time cut)', savefig_path=dir_path, fmt='png')
    # highlight the sensors
#     plot(df[S], "2D_Sensors", dict_of_batches[this_batch], S, bins=these_bins, bins_find_min=binning_method,
#          n_DUT=DUTs, savefig=SAVE, savefig_details=f' {S} (pulseHeight cut)', savefig_path=dir_path, fmt='png')
    ## delta time vs pulseHeight w/ info
    plot(df[S], "Time_pulseHeight", dict_of_batches[this_batch], S, bins=time_bins,
         n_DUT=DUTs, savefig=SAVE, savefig_details=f' {S} ', savefig_path=dir_path, fmt='png')
    ### delta time vs pulseHeight no info
    plot(df[S], "Time_pulseHeight", dict_of_batches[this_batch], S, bins=time_bins, info=False, extra_info=False,
         n_DUT=DUTs, savefig=SAVE, savefig_details=f' {S} no info', savefig_path=dir_path, fmt='png') 
    ### delta time vs pulseHeight no info
    plot(df[S], "Time_pulseHeight", dict_of_batches[this_batch], S, bins=time_bins, info=False, extra_info=False, mask=central_sensor_area_cuts,
         n_DUT=DUTs, savefig=SAVE, savefig_details=f' {S} central area', savefig_path=dir_path, fmt='png')
    ### efficiency projection whole sensor (zooomed)
    plot(df[S], "1D_Efficiency", dict_of_batches[this_batch], S, threshold_charge=threshold_charge, transimpedance=my_transimpedance, geometry_cut='normal', use=use_for_geometry_cut, zoom_to_sensor=True, efficiency_lim=(0.4,1),
        bins=these_bins, bins_find_min=binning_method, n_DUT=DUTs, savefig=SAVE, savefig_details=f' {S} threshold charge {threshold_charge}fC', savefig_path=dir_path)
    ### efficiency projection in the center (zoomed)
    plot(df[S], "1D_Efficiency", dict_of_batches[this_batch], S, threshold_charge=threshold_charge, transimpedance=my_transimpedance, geometry_cut='center', use=use_for_geometry_cut, zoom_to_sensor=True, efficiency_lim=(0.4,1),
        bins=these_bins, bins_find_min=binning_method, n_DUT=DUTs, savefig=SAVE, savefig_details=f' {S} threshold charge {threshold_charge}fC (center)', savefig_path=dir_path)
    ### with time cut (zoomed)
    plot(df[S], "1D_Efficiency", dict_of_batches[this_batch], S, threshold_charge=threshold_charge, transimpedance=my_transimpedance, geometry_cut='normal', use=use_for_geometry_cut, mask=time_cut, zoom_to_sensor=True, efficiency_lim=(0.4,1),
        bins=these_bins, bins_find_min=binning_method, n_DUT=DUTs, savefig=SAVE, savefig_details=f' {S} threshold charge {threshold_charge}fC (time cut)', savefig_path=dir_path)
    ### with time cut in the center (zoomed)
    plot(df[S], "1D_Efficiency", dict_of_batches[this_batch], S, threshold_charge=threshold_charge, transimpedance=my_transimpedance, geometry_cut='center', use=use_for_geometry_cut, mask=time_cut, zoom_to_sensor=True, efficiency_lim=(0.4,1),
        bins=these_bins, bins_find_min=binning_method, n_DUT=DUTs, savefig=SAVE, savefig_details=f' {S} threshold charge {threshold_charge}fC (center and time cut)', savefig_path=dir_path)
    ### 2D efficiency
    plot(df[S], "2D_Efficiency", dict_of_batches[this_batch], S, threshold_charge=threshold_charge, transimpedance=my_transimpedance, geometry_cut='normal', use=use_for_geometry_cut, zoom_to_sensor=True,
        bins=these_bins, bins_find_min=binning_method, n_DUT=DUTs, savefig=SAVE, savefig_details=f' {S} thresh charge {threshold_charge}fC', savefig_path=dir_path, fmt='png')
    ### with time cut and zoomed
    plot(df[S], "2D_Efficiency", dict_of_batches[this_batch], S, threshold_charge=threshold_charge, transimpedance=my_transimpedance, geometry_cut='normal', use=use_for_geometry_cut, mask=time_cut, zoom_to_sensor=True,
        bins=these_bins, bins_find_min=binning_method, n_DUT=DUTs, savefig=SAVE, savefig_details=f' {S} thresh charge {threshold_charge}fC (w time cut)', savefig_path=dir_path, fmt='png')
    
    ### I NEED TO ADD THE PLOT OF THE CHARGE TOO

    for dut in DUTs:
        fig, axes = plt.subplots(figsize=(6*axes_size,4*axes_size), nrows=axes_size, ncols=axes_size, dpi=300)

        time_resolution_table = []
        chi2_table = []

        for i, ax in enumerate(axes.flatten()):
            CFD_MCP = CFD_values[i//axes_size]
            CFD_DUT = CFD_values[i%axes_size]

            window_fit = np.logical_and((df[S][f"timeCFD{CFD_DUT}_{dut}"]-df[S][f"timeCFD{CFD_MCP}_0"])> -window_limit,
                                       (df[S][f"timeCFD{CFD_DUT}_{dut}"]-df[S][f"timeCFD{CFD_MCP}_0"])< +window_limit)
            # dut_cut = np.logical_and(window_fit, all_cuts[dut-1])
            # dut_cut = np.logical_and(window_fit, np.logical_and(pulse_cuts[dut-1],geo_cuts[dut-1]))
            ### ONLY EVENTS WITH CHARGE OVER THE THRESHOLD CHARGE
            dut_cut = np.logical_and(df[S][f"charge_{dut}"]>threshold_charge,
                                     np.logical_and(window_fit, np.logical_and(pulseheight_cut[dut-1],geo_cuts[dut-1])))

            hist, my_bins,_,_,_ = plot_histogram((df[S][f"timeCFD{CFD_DUT}_{dut}"].loc[dut_cut]-df[S][f"timeCFD{CFD_MCP}_0"].loc[dut_cut]),
                                                 bins=time_bins, color='k', linewidth=1, alpha=1,
                                                 fig_ax=(fig,ax))

            bins_centers = (my_bins[:-1]+my_bins[1:])/2
            initial_param = (np.max(hist),bins_centers[np.argmax(hist)],100,np.average(hist))
            param, covar = curve_fit(my_gauss, bins_centers, hist, p0=initial_param)#, sigma=hist**0.5, absolute_sigma=True)
            #     print(f"Fit parameters: {param}"
            ax.plot(bins_centers, my_gauss(bins_centers,*param), color=colormap[dut])

            ### add units to the parameters
            ax.plot([],[],linewidth=0, label="A: %.0f" %param[0]) # only two decimals
            ax.plot([],[],linewidth=0, label="$\mu$: %.1f $\pm$ %.1f"%(param[1],covar[1,1]**0.5))
            ax.plot([],[],linewidth=0, label="$\sigma$: %.2f $\pm$ %.2f"%(param[2],covar[2,2]**0.5))
            ax.plot([],[],linewidth=0, label="BG: %.1f $\pm$ %.1f"%(param[3],covar[3,3]**0.5))
            chi2_reduced = sum((hist-my_gauss(bins_centers,*param))**2/my_gauss(bins_centers,*param))/(len(hist)-len(param))
            ax.plot([],[],linewidth=0, label="$\chi^2$ reduced: "+f"%.1f"%chi2_reduced)
            ### skewness doesn't work, idk why
        #     skewness = skew((df[S][f"timeCFD{CFD_DUT}_{dut}"].loc[dut_cut]-df[S][f"timeCFD{CFD_MCP}_0"].loc[dut_cut]))
        #     ax.plot([],[],linewidth=0, label="skewness: %.1f" %skewness[1])

        #     ax.plot([],[],linewidth=0, label=f"(MCP: CFD{CFD_MCP}%, DUT: CFD{CFD_DUT}%)")
            ax.set_title(f"MCP: CFD{CFD_MCP}%, DUT: CFD{CFD_DUT}%", fontsize=16)
            ax.set_xlabel(f"$\Delta t$ [ps]", fontsize=16)
            ax.set_ylabel("Events", fontsize=16)
            time_resolution_table.append(np.sqrt(param[2]**2-MCP_resolution**2))
            chi2_table.append(chi2_reduced)

#             xlim = (-7e3,-5e3)
            xlim = (-6.5e3,-4.5e3)
            ax.set_xlim(xlim)
            ax.legend(fontsize=16, framealpha=0, markerscale=0)


        fig.tight_layout(w_pad=4, h_pad=4)
        sensor_name = dict_of_batches[this_batch].S[S].get_sensor(f'Ch{dut+1}').name
        fig.suptitle(f"Time resolution fit, after applying cuts \
        \n Batch: {this_batch}, Oscilloscope: {S}, Ch{dut+1}: {sensor_name}",y=1.1 , fontsize=20)

        if SAVE:
            fig.savefig(os.path.join(dir_path, f"time_resolution_{this_batch}_{S}_dut{dut}_comparing CFD.png"), bbox_inches="tight")
    plt.close('all')

[INFO] - 	 Loading batch 501 	 Oscilloscope S1


Batch:  501
S1
MCP: 2800 V, angle: 0.0° temperature:-26.57°C


[INFO] - 	 in 'time_mask()': Fit parameters [ 3.80149957e+04 -1.46091036e+02 -7.05638373e+00  6.87386732e+00]
[INFO] - 	 in 'time_mask()': Fit parameters [ 9646.20866911 -6186.97035897   115.05367392   162.47062312]
[INFO] - 	 in 'time_mask()': Fit parameters [20529.74630881 -6143.00269278    61.83093965   154.80964155]
[INFO] - 	 in 'time_mask()': Fit parameters [ 3.80149957e+04 -1.46091036e+02 -7.05638373e+00  6.87386732e+00]
[INFO] - 	 in 'time_mask()': Fit parameters [ 9646.20866911 -6186.97035897   115.05367392   162.47062312]
[INFO] - 	 in 'time_mask()': Fit parameters [20529.74630881 -6143.00269278    61.83093965   154.80964155]
[INFO] - 	 in 'time_mask()': Fit parameters [  103.59613532 -6189.44677512   113.82074473    25.1864695 ]
[INFO] - 	 in 'time_mask()': Fit parameters [10561.29499795 -6142.2941828     61.72741247    21.55049781]
[INFO] - 	 in 'time_mask()': Fit parameters [  306.69712657 -6011.53812519    98.91258909    54.88395473]
[INFO] - 	 Two peaks not found, retryi

In [ ]:
### TIME RESOLUTION PLOTS (all sensors together)
SAVE = True

colormap = ['k','b','g','r']

threshold_charge = 4 #fC
MCP_resolution = 36.52

CFD_values = (20, 50,70)
axes_size = len(CFD_values)
window_limit = 20e3

for S in ['S1','S2']:
    DUTs = get_DUTs_from_dictionary(info_dict,S)
    
    fig, axes = plt.subplots(figsize=(12,8), nrows=1, ncols=1, dpi=300)
    
    geo_cuts = [geometry_mask(df[S], DUT_number=dut, bins=these_bins, bins_find_min='rice')[0] if dut in DUTs else None for dut in [1,2,3]]
#     central_sensor_area_cuts = [geometry_mask(df[S], DUT_number=dut, bins=these_bins, bins_find_min='rice', only_select='center')[0] if dut in DUTs else None for dut in [1,2,3]]
#     time_cut = [time_mask(df[S], dut, bins=time_bins, mask=geo_cuts[dut-1], plot=False, savefig=os.path.join(dir_path,f'time_plot_with_geo_cuts_{S}_{this_batch}_DUT{dut}.png'))[0] if dut in DUTs else None for dut in [1,2,3]]
    mins = [find_min_btw_peaks(df[S][f"pulseHeight_{dut}"], bins='rice', plot=False) if dut in DUTs else None for dut in [1,2,3]]
    pulseheight_cut = [df[S][f'pulseHeight_{dut}']>mins[dut-1] if dut in DUTs else None for dut in [1,2,3]]
    charge_cut = [df[S][f'charge_{dut}']>threshold_charge if dut in DUTs else None for dut in [1,2,3]]
    pulse_geo_cuts = [np.logical_and(geo_cuts[dut-1], pulseheight_cut[dut-1]) for dut in DUTs]

    for dut in DUTs:
        window_fit = np.logical_and((df[S][f"timeCFD20_{dut}"]-df[S]["timeCFD50_0"])> -window_limit,
                                   (df[S][f"timeCFD20_{dut}"]-df[S]["timeCFD50_0"])< +window_limit)
    #     dut_cut = np.logical_and(window_fit, all_cuts[dut-1])
    #     dut_cut = np.logical_and(window_fit, np.logical_and(pulseheight_cut[dut-1],geo_cuts[dut-1]))

    ### ONLY EVENTS WITH CHARGE OVER THE THRESHOLD CHARGE
        dut_cut = np.logical_and(charge_cut[dut-1],
                                 np.logical_and(window_fit, np.logical_and(pulseheight_cut[dut-1],geo_cuts[dut-1])))

        hist, my_bins,_,_,_ = plot_histogram((df[S][f"timeCFD20_{dut}"].loc[dut_cut]-df[S]["timeCFD50_0"].loc[dut_cut]),
                                             bins=time_bins, color='k', linewidth=1, alpha=1,
                                             fig_ax=(fig,axes))

        bins_centers = (my_bins[:-1]+my_bins[1:])/2
        initial_param = (np.max(hist),bins_centers[np.argmax(hist)],100,np.average(hist))
        param, covar = curve_fit(my_gauss, bins_centers, hist, p0=initial_param)#, sigma=hist**0.5, absolute_sigma=True)
    #     print(f"Fit parameters: {param}"
        axes.plot(bins_centers, my_gauss(bins_centers,*param), color=colormap[dut])

        ### add units to the parameters
        axes.plot([],[],linewidth=3, label=f"{dict_of_batches[this_batch].S[S].get_sensor(f'ch{dut+1}').name}", color=colormap[dut])
    #     axes.plot([],[],linewidth=0, label="A: %.0f" %param[0]) # only two decimals
        axes.plot([],[],linewidth=0, label="$\mu$: %.1f $\pm$ %.1f"%(param[1],covar[1,1]**0.5))
        axes.plot([],[],linewidth=0, label="$\sigma$: %.1f $\pm$ %.1f"%(param[2],covar[2,2]**0.5))
    #     axes.plot([],[],linewidth=0, label="BG: %.1f $\pm$ %.1f"%(param[3],covar[3,3]**0.5))
        ### maybe I should just make the chi² into a function
        chi2_reduced = sum((hist-my_gauss(bins_centers,*param))**2/my_gauss(bins_centers,*param))/(len(hist)-len(param))
        axes.plot([],[],linewidth=0, label="$\chi^2$ reduced: "+f"%.1f"%chi2_reduced)
    #     skewness = skew(df[S][f"timeCFD20_{dut}"].loc[dut_cut]-df[S]["timeCFD50_0"].loc[dut_cut])
    #     axes.plot([],[],linewidth=0, label=f"skewness: {skewness}")

    axes.set_xlabel(f"$\Delta t$ [ps]", fontsize=16)
    axes.set_ylabel("Events", fontsize=16)

    xlim = (-6.5e3,-4.5e3)
    # xlim = (-10e3,0)
    axes.set_xlim(xlim)
    axes.legend(fontsize=16)

    fig.suptitle(f"Time resolution fit, after applying cuts \
    \n Batch: {this_batch}, Oscilloscope: {S}",y=1, fontsize=20)

    if SAVE:
        fig.savefig(os.path.join(dir_path, f"time_resolution_{this_batch}_{S}_zoomed_and_gauss_fit_with_cuts.png"), bbox_inches="tight")

In [25]:
{100:bins3, # x
 101:bins3, # x
 199:bins3, # x
 201:bins1, # x
 202:bins1, # x
 203:bins1, # x
 204:bins1, # x
 205:bins4, # x
 206:bins4, # x
 301:bins1, # x
 401:bins1, # x
 402:bins1, # x
 403:bins1, # x
 407:bins1, # x
 408:bins1, # x
 409:bins1, # x
 410:bins1, # x
 411:bins1, # x
 413:bins1, # x
 414:bins1, # x
 501:bins2, #
 502:bins2, #
 503:bins2, #
 504:bins2, #
 505:bins2, #
 601:bins1, #
 602:bins1, #
 603:bins1, #
 604:bins1, #
 605:bins1, #
 701:bins1, #
 702:bins1, #
 801:bins1, #
 802:bins1, #
 901:bins1, #
 902:bins1, #
 1001:bins1, #
 1002:bins1, #
 1101:bins1, #
 1102:bins1, #

}

{100: (array([300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312,
         313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325,
         326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338,
         339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351,
         352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364,
         365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377,
         378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390,
         391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403,
         404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416,
         417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429,
         430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442,
         443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455,
         456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468,
         469, 470, 471, 472, 473,

In [ ]:
# pulseHeight_filter = df['S2']['pulseHeight_2']>mins['S2'][2]
# x_cut = df['S2']['Xtr_1'].loc[pulseHeight_filter]
# find_edges(x_cut, bins=my_bins[0], plot=True)

## Calculate all minimums (for this batch)
often useful

In [ ]:
mins = {}
for S in ["S1","S2"]:
    mins[S] = [find_min_btw_peaks(df[S][f"pulseHeight_{i}"], bins='rice', plot=False) for i in range(1,4)]
    mins[S].insert(0,None)     ### insert None as the 'peak' of the MCP

# PulseHeight vs $\Delta$t plot
I should try:  
https://github.com/astrofrog/mpl-scatter-density

#### PLOT ALL BATCHES (might be slow, can be done in parts)
I don't think this is a good thing to do, I should just save all the results while I do them

In [ ]:
# threshold_charge = 4

# ### SAVE THE PLOTS FOR ALL BATCHES
# binning_method = 'rice'
# RUN_ALL_BATCHES = False  ### so I don't accidentally run this
# if RUN_ALL_BATCHES:
#     for S in ["S1","S2"]: ### the two scopes
#         if S=="S2": continue
#         for this_batch in np.sort(list(dict_of_runs[S].keys())):  ### I sort them so I can restart from the batch I want
#             if this_batch>=0:# and this_batch<=500:
#                 print("Batch: ", this_batch)
#                 dir_path = f"../Data_TestBeam/2023_May/{S}/"
#                 file_path = f"tree_May2023_{S}_{this_batch}.root"    
#                 try:
#                     df = root_to_df(os.path.join(dir_path, file_path), branches)
#                 except:
#                     print("something wrong reading the file")
#                     continue
#                 df = df.drop(columns=columns_to_remove)
# #                 plot(df,"2D_Tracks", dict_of_batches[this_batch], S, bins=bins,
#         #              savefig=True, savefig_details=f'_{S}')
# #                 plot(df,"1D_Tracks", dict_of_batches[this_batch], S, bins=large_bins,
#     #                  savefig=True, savefig_details=f'_{S}')
#         #         bins = 1000
# #                 plot(df, "pulseHeight", dict_of_batches[this_batch], S, bins=bins,
#         #             savefig=True, savefig_details=f"_{S}_bins_{bins}")
# #                 plot(df, '2D_Sensors', dict_of_batches[this_batch], S,
# #                      bins=large_bins, bins_find_min=binning_method, savefig=False, savefig_details=f'_{S}_large_bins')#_{binning_method}')
# #                 plot(df, "1D_Efficiency", dict_of_batches[this_batch], S, threshold_charge=threshold_charge, geometry_cut='normal',
# #                     bins=large_bins, bins_find_min=binning_method, savefig=True, savefig_details=f'_{S}_threshold_charge_{threshold_charge}fC_large_bins')
# #                 plot(df, "2D_Efficiency", dict_of_batches[this_batch], S, threshold_charge=threshold_charge, geometry_cut='normal',
# #                     bins=large_bins, bins_find_min=binning_method, savefig=True, savefig_details=f'_{S}_thresh_charge_{threshold_charge}fC_large_bins')
#             ### I think something is leaking memory!!!
#             plt.close('all') ### I think this worked. maybe not actually


## Plot a single run

In [ ]:
### looking at single runs 
S_run = 'S1'
dir_path = f"../Data_TestBeam/2023_May/{S_run}/"
run = 9734

if run>9999:    file_path = f"tree0{run}.root"
elif run<=9999: file_path = f"tree00{run}.root"

### branches to be loaded (+ unnecessary columns)
branches = ["eventNumber", "Xtr", "Ytr", "pulseHeight", "charge", "timeCFD20", "timeCFD50", "timeCFD70"]
columns_to_remove = ["Xtr_4","Xtr_5","Xtr_6","Xtr_7","Ytr_4","Ytr_5","Ytr_6","Ytr_7"]

try:
    df_run = root_to_df(os.path.join(dir_path, file_path), branches)
except FileNotFoundError:
    logging.error("Batch file not found")

df_run = root_to_df(os.path.join(dir_path, file_path), branches)
df_run = df_run.drop(columns=columns_to_remove)
    
plot(df_run,"2D_Sensors", dict_of_batches[412], this_scope='S1', bins=large_bins,
    savefig=False, savefig_details=f"_single_run_{run}")
# plot(df["S2"],"2D_Tracks", batch=this_batch, sensors=sensors_list[this_batch]["S2"], bins=bins)

#### Improved efficiency study

In [ ]:
this_scope = "S1"
threshold_charge = 4
plot(df[this_scope], "1D_Efficiency", dict_of_batches[this_batch], this_scope, bins=bins, geometry_cut='normal', n_DUT=[1,2,3],
                 threshold_charge=threshold_charge,
                 savefig=False, savefig_details=f'_{this_scope}_charge: {threshold_charge}fC')

# fig.savefig("../various plots/1D_Efficiency_410_S1_charge: 2fC.svg")

In [ ]:
fig, axes = plot(df[this_scope], "2D_Efficiency", dict_of_batches[this_batch], this_scope, bins=bins, geometry_cut='normal', zoom_to_sensor=True,
                 n_DUT=[1,2,3], threshold_charge=threshold_charge, savefig=False)

In [ ]:
# print(sensors_list[this_batch][this_scope]['Ch2'])
# print(dict_of_batches[this_batch].S[this_scope].channels['Ch2'].name)

In [ ]:
# fig, ax = plot(df[this_scope], "2D_Tracks",batch=this_batch, bins=large_bins)
# fig.suptitle("2D_Tracks, batch: 410", y=0.95)
# # fig

## Efficiency Study

In [ ]:
# ### bin per bin efficiency
# threshold_charge = 4 #fC
# # transimpedance = dict_of_batches[this_batch].S[this_scope].channels[]

# fig,axes = plot(df[this_scope], "2D_Efficiency", dict_of_batches[this_batch], this_scope, bins=bins, geometry_cut=False,
#                 threshold_charge=threshold_charge, savefig=False, savefig_details=" (no cut)")
# # fig

### plotting where the noise is
aka where pulseHeight<mins.
This shows how 'good' cutting the PulseHeight is, so where the events that have pulseHeight<mins lie

In [ ]:
# # this_scope = "S2"
# mask = [df[this_scope][f"pulseHeight_{i}"]<mins[this_scope][i] for i in range(1,4)]

# fig,axes = plot(df[this_scope], "2D_Tracks", dict_of_batches[this_batch], this_scope, bins=bins, mask=mask)

## Total efficiency with pulseHeight cut

In [ ]:
# this_scope = "S1"
# dut = 1
# transimpedance = get_transimpedance(this_batch, this_scope) ### 4700 or  10700
# threshold_range = np.arange(0.5, 20, 0.5)

# ### full surface and pulseHeight cut
# efficiency_range = []
# error_range = []
# geometry = geometry_mask(df[this_scope], bins=bins, bins_find_min='rice', DUT_number=dut)[0]
# pulseHeight = df[this_scope][f"pulseHeight_{dut}"]>mins[this_scope][dut]
# for charge in threshold_range:
#     ### efficiency with error
#     efficiency_range.append(efficiency_error(
#         df[this_scope][f"charge_{dut}"].loc[np.logical_and(geometry,pulseHeight)]/transimpedance[dut-1], charge)[0])
#     error_range.append(efficiency_error(
#         df[this_scope][f"charge_{dut}"].loc[np.logical_and(geometry,pulseHeight)]/transimpedance[dut-1], charge)[1]) 

# fig, ax = plt.subplots(figsize=(10,6), dpi=150)
# ax.errorbar(threshold_range, efficiency_range, yerr=error_range, marker='.', markersize=3, linewidth=0,
#             elinewidth=1, ecolor='k', capsize=1.5,
#         label=f"Ch{dut+1}, Total tracks: {df[this_scope][f'charge_{dut}'].loc[geometry].size}")

# ax.set_title(f"Total efficiency depending on threshold charge \n \
# on full surface of Ch{dut+1}: {sensors_list[this_batch][this_scope][f'Ch{dut+1}']}, batch {this_batch} \n \
# (with pulseHeight cut)")
# ax.set_xlabel("Threshold charge (fC)")
# ax.set_ylabel("Total efficiency")
# ax.axhline(0.95, label="95% efficiency", color='r', alpha=0.4, linewidth=2)
# ax.grid('--')
# ax.legend()
# # fig.savefig(f"..\various plots\Total_efficiency_change_{this_batch}_{this_scope}_DUT{dut}_pulseheight_cut.svg")


## Efficiency in the central part 
Calculate the efficiency in the central $0.5x0.5$ $mm^2$

In [ ]:
# mask = [geometry_mask(df[this_scope], DUT_number=i, bins=bins, bins_find_min='rice', only_select='center')[0] for i in DUTs]

# fig,axes = plot(df[this_scope], "2D_Tracks", dict_of_batches[this_batch], this_scope, bins=bins, mask=mask)

In [ ]:
# fig, axes = plot(df[this_scope],"2D_Efficiency", dict_of_batches[this_batch], this_scope, bins=bins, threshold_charge=threshold_charge,
#                 geometry_cut='center', zoom_to_sensor=True, savefig=False, savefig_details=f"_{this_scope}_central_area")

## Ratio plots

In [ ]:
# ### THIS IS PLOTTING THE RATIO OF DATA THAT IS LEFT WHEN APPLYIN A PULSEHEIGHT CUT
# # this_scope = "S2"

# colormap = ['k','b','g','r']

# fig, axes = plt.subplots(figsize=(15,8),nrows=1,ncols=2, dpi=300)
# fig.tight_layout(w_pad=4, h_pad=4)
# ax2 = axes[1].twinx() 
# ax2.grid('-', axis='y', color='r')

# for dut in range(1,4):
#     pulse_cut = df[this_scope][f"pulseHeight_{dut}"]>mins[this_scope][dut]
#     hist, my_bins,_,_,_ = plot_histogram((df[this_scope][f"timeCFD50_{dut}"]-df[this_scope]["timeCFD50_0"])/1000, bins=10000, label=f"$\Delta$t Ch{dut+1}-Ch1",fig_ax=(fig,axes[0]), color=colormap[dut]) ### MCP - CERN1
#     axes[0].plot([],[],' ',label=f"Ch{dut+1}: {dict_of_batches[this_batch].S[this_scope].get_sensor(f'Ch{dut+1}').name}")
#     hist_pulse_cut,_,_,_,_ = plot_histogram((df[this_scope][f"timeCFD50_{dut}"].loc[pulse_cut]-df[this_scope]["timeCFD50_0"].loc[pulse_cut])/1000, bins=my_bins, label=f"$\Delta$t Ch{dut+1}-Ch1 \n w\ pulseHeigh cut", fig_ax=(fig,axes[1]), linewidth=2, color=colormap[dut])  ### MCP - CERN3
# #     time_peak,_ = find_peaks(hist_pulse_cut, prominence=np.max(hist_pulse_cut)/10)
#     ratio = np.divide(hist_pulse_cut, hist, where = hist!=0)
# #     axes[0].stairs(ratio*100, my_bins, alpha=0.5, label='Filtered Ratio', color=colormap[dut])
#     ax2.stairs(ratio*100, my_bins, alpha=0.3, label=f"Ratio of events \n (Ch{dut+1}-Ch1)", color=colormap[dut], fill=True)


# axes[0].set_title(f"Time difference", fontsize=16)
# axes[0].set_xlabel("Time (ns)", fontsize=14)
# axes[0].set_ylabel("Events", fontsize=14)
# axes[0].set_xlim(-8,-2)
# # axes[0].semilogy()

# axes[1].set_title("w/ pulseHeight cut (and zoomed)", fontsize=16)
# axes[1].set_xlabel("Time (ns)", fontsize=14)
# axes[1].set_ylabel("Events", fontsize=14)
# axes[1].set_xlim(-6.5,-5)

# ax2.set_ylabel("Events(pulse_cut) \ Events (%)", fontsize=14)
# ax2.set_ylim(0,100)
# ax2.tick_params

# axes[0].legend(fontsize=16)
# axes[1].legend(fontsize=16)
# ax2.legend(fontsize=16, loc='right')
# fig.suptitle(f"Time difference with ratio plot when pulseHeight cut is applied \
# \n Batch: {this_batch}, Oscilloscope: {this_scope}",y=1.15, fontsize=20)
# # fig.savefig(f"../various plots/time_difference{this_batch}_{this_scope}_all_DUTs_with_ratio_plot.svg",bbox_inches='tight')

### New ratio plot
 I apply a pulseHeight cut and then I use __find_peaks__ to find the peaks

In [ ]:
# ### THIS IS PLOTTING THE RATIO OF DATA THAT IS LEFT WHEN APPLYIN A PULSEHEIGHT CUT

# # this_scope = "S2"

# colormap = ['k','b','g','r']
# spec = {'height_ratios':[4,4,1], 'hspace':0.2}   # gridspec keywords arguments
# fig, axes = plt.subplots(figsize=(12,15),nrows=3, ncols=1, gridspec_kw=spec, sharex=True, dpi=300)

# # all_time_cut1 = [None]

# for dut in range(1,4):
#     hist, my_bins,_,_,_ = plot_histogram((df[this_scope][f"timeCFD50_{dut}"]-df[this_scope]["timeCFD50_0"]),
#                                          bins=10000, fig_ax=(fig,axes[1]), color=colormap[dut], linewidth=0) #,label=f"$\Delta$t Ch{dut+1}-Ch1",) ### MCP - CERN1
#     axes[0].plot([],[],' ',label=f"Ch{dut+1}: {sensors_list[this_batch][this_scope][f'Ch{dut+1}']}")
#     if mins[this_scope][dut]:
#         pulse_cut = df[this_scope][f"pulseHeight_{dut}"]>mins[this_scope][dut]
#         hist_pulse_cut,_,_,_,_ = plot_histogram((df[this_scope][f"timeCFD50_{dut}"].loc[pulse_cut]-df[this_scope]["timeCFD50_0"].loc[pulse_cut]),
#                                                 poisson_err=True, error_band=True, bins=my_bins, color=colormap[dut],
#                                                 fig_ax=(fig,axes[0]), label=f"$\Delta$t Ch{dut+1}-Ch1 \n w/ pulseHeigh cut")  ### MCP - CERN3
#         peaks, info_peaks = find_peaks(hist_pulse_cut, prominence=np.max(hist_pulse_cut)/10)
#     else:
#         print(f"DUT: {dut} Try without pulseheight cut")
#         peaks, info_peaks = find_peaks(hist, prominence=np.max(hist)/10)
        
#     left_cut = (df[this_scope][f"timeCFD50_{dut}"]-df[this_scope]["timeCFD50_0"])>my_bins[info_peaks['left_bases'][0]]
#     right_cut = (df[this_scope][f"timeCFD50_{dut}"]-df[this_scope]["timeCFD50_0"])<my_bins[info_peaks['right_bases'][0]]
# #     all_time_cut1.insert(dut, np.logical_and(left_cut, right_cut))
#     time_cut = np.logical_and(left_cut, right_cut)
#     hist_time_cut,_,_,_,_ = plot_histogram((df[this_scope][f"timeCFD50_{dut}"].loc[time_cut]-df[this_scope]["timeCFD50_0"].loc[time_cut]),
#                                            poisson_err=True, error_band=True, bins=my_bins, linewidth=1, color=colormap[dut],
#                                            fig_ax=(fig,axes[1]), label=f"$\Delta$t Ch{dut+1}-Ch1 \n w/ time cut")  ### MCP - CERN3
#     ratio = np.divide(hist_pulse_cut, hist_time_cut, where = hist_time_cut!=0)
#     axes[2].stairs(ratio, my_bins, label=f"Ratio of events \n (Ch{dut+1}-Ch1)", color=colormap[dut])#, fill=True)

    
# axes[0].set_title(f"Time difference", fontsize=16)
# axes[0].set_ylabel("Events", fontsize=14)
# # axes[0].set_xlim(-7,-5)
# axes[0].set_xlim(-7e3,-4e3)
# # axes[0].semilogy()

# axes[2].set_ylabel("Ratio of events", fontsize=14)
# axes[2].set_xlabel("Time (ps)", fontsize=14)

# axes[2].set_title("Events(pulse_cut) / Events(time_cut) in each bin", fontsize=16)

# axes[0].legend(fontsize=16)
# axes[1].legend(fontsize=16)
# axes[2].grid('-')

# fig.suptitle(f"Time difference with ratio plot when pulseHeight cut is applied \
# \n Batch: {this_batch}, Oscilloscope: {this_scope}",y=1, fontsize=20)
# # fig.savefig(f"../various plots/time_difference_pulseheight_cut_time_cut_{this_batch}_{this_scope}_all_DUTs_with_ratio_plot.svg",bbox_inches='tight')

### Try to define the time cut without other info. And fitting the gaussian + background
i.e. when pulseHeight cut (and so geometry cut) are not available

In [ ]:
all_time_cut = [time_mask(df[this_scope], bins=5000, DUT_number=i, plot=False)[0] for i in [1,2,3]]

In [ ]:
# ### THIS IS PLOTTING THE RATIO OF DATA THAT IS LEFT WHEN APPLYIN A PULSEHEIGHT CUT
# # this_scope = "S2"

# colormap = ['k','b','g','r']
# spec = {'height_ratios':[4,4,1], 'hspace':0.2}   # gridspec keywords arguments

# fig, axes = plt.subplots(figsize=(12,15),nrows=3, ncols=1, gridspec_kw=spec, sharex=True, dpi=300)

# for dut in [1,2]:
#     hist, my_bins,_,_,_ = plot_histogram(df[this_scope][f"timeCFD50_{dut}"]-df[this_scope]["timeCFD20_0"],
#                                          bins=10000, color=colormap[dut], linewidth=1,
#                                          fig_ax=(fig,axes[0]))
    
#     axes[0].plot([],[],' ',label=f"Ch{dut+1}: {sensors_list[this_batch][this_scope][f'Ch{dut+1}']}")
#     initial_param = (np.max(hist),-5e3,100,100)
#     bins_centers = (my_bins[:-1]+my_bins[1:])/2
#     param, covar = curve_fit(my_gauss, bins_centers, hist, p0=initial_param)
#     print(f"Fit parameters: {param}")
#     axes[0].plot(bins_centers, my_gauss(bins_centers,*param), color='k')
#     number_of_sigmas = 5
#     left_base, right_base = param[1]-number_of_sigmas*param[2], param[1]+number_of_sigmas*param[2]
#     axes[0].axvline(left_base, color=colormap[dut])
#     axes[0].axvline(right_base, color=colormap[dut])
#     left_cut = (df[this_scope][f"timeCFD50_{dut}"]-df[this_scope]["timeCFD20_0"])>left_base
#     right_cut = (df[this_scope][f"timeCFD50_{dut}"]-df[this_scope]["timeCFD20_0"])<right_base
#     time_cut = np.logical_and(left_cut, right_cut)
#     hist_time_cut,_,_,_,_ = plot_histogram(df[this_scope][f"timeCFD50_{dut}"].loc[time_cut]-df[this_scope]["timeCFD20_0"].loc[time_cut],
#                                            poisson_err=True, error_band=True, bins=my_bins, linewidth=1, color=colormap[dut],
#                                            fig_ax=(fig,axes[1]), label=f"$\Delta$t Ch{dut+1}-Ch1 \n w/ time cut")  ### MCP - CERN3

#     ratio = np.divide(hist_time_cut, hist, where = hist_time_cut!=0)
#     axes[2].stairs(ratio, my_bins, label=f"Ratio of events \n (Ch{dut+1}-Ch1)", color=colormap[dut])#, fill=True)

    
# axes[0].set_title(f"Time difference", fontsize=16)
# axes[0].set_ylabel("Events", fontsize=14)
# axes[0].set_xlim(-6e3,-5e3)
# axes[1].set_xlim(-6e3,-5e3)

# axes[2].set_ylabel("Ratio of events", fontsize=14)
# axes[2].set_xlabel("Time (ns)", fontsize=14)
# axes[2].set_title("Events(time_cut) / Events(no_cut)", fontsize=16)

# axes[0].legend(fontsize=16)
# axes[1].legend(fontsize=16)
# axes[2].grid('-')

# fig.suptitle(f"Time difference with ratio plot when pulseHeight cut is applied \
# \n Batch: {this_batch}, Oscilloscope: {this_scope}",y=1, fontsize=20)

In [ ]:
# def my_gauss(x, A, mu, sigma, background):
#     """Custom normal distribution function + uniform background"""
#     return A * np.exp(-0.5*(x-mu)**2/sigma**2) + background

In [ ]:
# ### SAME THING BUT ONLY FIRST PLOT
# ### now I fit between -15ns < delta t < 15ns
# colormap = ['k','b','g','r']

# fig, axes = plt.subplots(figsize=(12,8),nrows=1, ncols=1, sharex=True, dpi=300)

# for dut in [1,2,3]:
#     window_fit = np.logical_and((df[this_scope][f"timeCFD50_{dut}"]-df[this_scope]["timeCFD20_0"])>-2e4,
#                                (df[this_scope][f"timeCFD50_{dut}"]-df[this_scope]["timeCFD20_0"])<2e4)
    
#     hist, my_bins,_,_,_ = plot_histogram((df[this_scope][f"timeCFD50_{dut}"].loc[window_fit]-df[this_scope]["timeCFD20_0"].loc[window_fit]),
#                                          bins=5000, color=colormap[dut], linewidth=1, alpha=0.7,
#                                          fig_ax=(fig,axes))
    
# #     hist, my_bins,_,_,_ = plot_histogram(df[this_scope][f"timeCFD50_{dut}"]-df[this_scope]["timeCFD20_0"],
# #                                          bins=10000, color=colormap[dut], linewidth=1, alpha=0.7,
# #                                          fig_ax=(fig,axes))
#     bins_centers = (my_bins[:-1]+my_bins[1:])/2
#     initial_param = (np.max(hist),bins_centers[np.argmax(hist)],-100,np.average(hist))
#     param, covar = curve_fit(my_gauss, bins_centers, hist, p0=initial_param)
#     print(f"Fit parameters: {param}")
#     axes.plot(bins_centers, my_gauss(bins_centers,*param), color='k',
#               label=f"Ch{dut+1}: {dict_of_batches[this_batch].S[this_scope].get_sensor(f'Ch{dut+1}').name}")
#     number_of_sigmas = 4
#     left_base, right_base = param[1]-number_of_sigmas*param[2], param[1]+number_of_sigmas*param[2]
# #     axes.axvline(left_base, color=colormap[dut])
# #     axes.axvline(right_base, color=colormap[dut])

#     axes.plot([],[],color=colormap[dut], label="A: %.0f, $\mu$: %.0f, $\sigma$: %.0f, BG: %.0f" %(param[0],param[1], param[2], param[3])) # only two decimals
    
# axes.set_xlabel(f"$\Delta t$ [ps]", fontsize=16)
# axes.set_ylabel("Events", fontsize=16)
# # axes.set_xlim(-10e3,-2e3)

# axes.legend(fontsize=16)

# fig.suptitle(f"Time difference \
# \n Batch: {this_batch}, Oscilloscope: {this_scope}",y=1, fontsize=20)
# # fig.savefig(f"../various plots/time_difference_{this_batch}_{this_scope}_small_range_with_fit.svg")

In [ ]:
# fig, axes = plt.subplots(figsize=(12,8),nrows=1, ncols=1, sharex=True, dpi=300)

# for dut in [1,2,3]:
#     hist, my_bins,_,_,_ = plot_histogram(df[this_scope][f"timeCFD50_{dut}"]-df[this_scope]["timeCFD20_0"],
#                                          bins=10000, color=colormap[dut], linewidth=1, label=f"Ch{dut+1}: {dict_of_batches[this_batch].S[this_scope].get_sensor(f'Ch{dut+1}').name}",
#                                          fig_ax=(fig,axes))
    
    
# axes.set_xlabel(f"$\Delta t$ [ps]", fontsize=16)
# axes.set_title(f"Time difference with gauss fit", fontsize=16)
# axes.set_ylabel("Events", fontsize=16)
# # axes.set_xlim(-10e3,-2e3)

# axes.legend(fontsize=16)

# fig.suptitle(f"Time difference \
# \n Batch: {this_batch}, Oscilloscope: {this_scope}",y=1, fontsize=20)

In [ ]:
# ### THIS TIME I'M PLOTTING THE CHARGE INSTEAD OF TIME
# # this_scope = "S2"
# # transimpedance = get_transimpedance(this_batch,this_scope)
# transimpedance = [dict_of_batches[this_batch].S[this_scope].get_sensor(f'ch{dut+1}').transimpedance for dut in (1,2,3)]

# # colormap = ['k','b','g','r']
# spec = {'height_ratios':[4,1], 'hspace':0.2}   # gridspec keywords arguments

# fig, axes = plt.subplots(figsize=(12,10),nrows=2, ncols=1, gridspec_kw=spec, sharex=True, dpi=300)

# # for dut in [1,2]:
# dut = 3
# hist, bins_charge,_,_,_ = plot_histogram(df[this_scope][f"charge_{dut}"]/transimpedance[dut-1],
#                                        bins='auto', linewidth=1, alpha=1, color='k',
#                                        fig_ax=(fig,axes[0]), label=f"No cut")  ### MCP - CERN3

# pulseheight_cut = df[this_scope][f"pulseHeight_{dut}"]>mins[this_scope][dut]
# hist_pulse_cut,_,_,_,_ = plot_histogram(df[this_scope][f"charge_{dut}"].loc[pulseheight_cut]/transimpedance[dut-1],
#                                         bins=bins_charge, alpha=1, color=colormap[1],
#                                         fig_ax=(fig,axes[0]), label=f"Pulseheight cut")

# time_cut = time_mask(df[this_scope], DUT_number=dut, bins=10000, plot=False)[0]
# hist_time_cut,_,_,_,_ = plot_histogram(df[this_scope][f"charge_{dut}"].loc[time_cut]/transimpedance[dut-1],
#                                        bins=bins_charge, alpha=1, color=colormap[2],
#                                    fig_ax=(fig,axes[0]), label=f"Time cut")

# ratio = np.divide(hist_pulse_cut, hist_time_cut, where = hist_time_cut!=0)
# axes[1].scatter((bins_charge[1:]+bins_charge[:-1])/2, ratio, .1, label=f"Events(pulse_cut) / Events(time_cut)",
#                 color='k', )#, fill=True)

# # ratio1 = np.divide(hist_pulse_cut, hist, where = hist!=0)
# # axes[1].scatter((bins_charge[1:]+bins_charge[:-1])/2, ratio1, .1, label=f"Events(pulse_cut) / Events(total)",
# #                 color=colormap[1], )#, fill=True)

# # ratio2 = np.divide(hist_time_cut, hist, where = hist!=0)
# # axes[1].scatter((bins_charge[1:]+bins_charge[:-1])/2, ratio2, .1, label=f"Events(time_cut) / Events(total)",
# #                 color=colormap[2], )#, fill=True)

    
# axes[0].set_title(f"Collected charge Ch{dut+1}", fontsize=20)
# axes[0].set_ylabel("Events", fontsize=16)
# axes[0].set_xlim(-10, 50)
# axes[0].semilogy()

# # axes[1].semilogy()
# # axes[1].set_xlim(-6e3,-5e3)
# # axes[1].semilogy()
# axes[1].grid('-')
# axes[1].set_ylabel("Ratio of events", fontsize=16)
# axes[1].set_xlabel("Charge (fC)", fontsize=16)
# axes[1].set_title(f"Ratio of events Ch{dut+1}", fontsize=16)

# axes[0].legend(fontsize=16)
# axes[1].legend(fontsize=16)

# fig.suptitle(f"Charge plot with two different cuts, with ratio\
# \n Batch: {this_batch}, Oscilloscope: {this_scope}",y=1, fontsize=24)
# # fig.savefig(f"../various plots/charge_plot_with_two_different_cuts_ratio_plot_batch_{this_batch}_{this_scope}_ch{dut+1}.svg")

### Efficiency with time_cut

In [ ]:
# # this_scope = "S2"
# # transimpedance = get_transimpedance(this_batch, this_scope) ### 4700 or  10700
# transimpedance = [dict_of_batches[this_batch].S[this_scope].get_sensor(f'ch{dut+1}').transimpedance for dut in (1,2,3)]

# dut = 2

# efficiency_range = []
# error_range = []
# threshold_range = np.arange(-5, 20, 0.5)
# geometry,edges = geometry_mask(df[this_scope], bins=bins, bins_find_min='rice', DUT_number=dut)
# # pulseHeight = df[this_scope][f"pulseHeight_{dut}"]>mins[this_scope][dut]

# for charge in threshold_range:
#     ### efficiency with error
#     eff, err = efficiency_error(
#         df[this_scope][f"charge_{dut}"].loc[all_time_cut[dut-1]]/transimpedance[dut-1], charge)
#     efficiency_range.append(eff)
#     error_range.append(err) 

# fig, ax = plt.subplots(figsize=(10,6), dpi=150)
# ax.errorbar(threshold_range, efficiency_range, yerr=error_range, marker='.', markersize=3, linewidth=0,
#             elinewidth=1, ecolor='k', capsize=1.5,
#         label=f"Total tracks: {df[this_scope][f'charge_{dut}'].loc[all_time_cut[dut-1]].size}")
# ax.set_title(f"Total efficiency depending on threshold charge \n \
# on full surface of {sensors_list[this_batch][this_scope][f'Ch{dut+1}']}, batch {this_batch} \n \
# (with time cut)")
# ax.set_xlabel("Threshold charge (fC)")
# ax.set_ylabel("Total efficiency")
# ax.axhline(0.95, label="95% efficiency", color='r', alpha=0.4, linewidth=2)
# ax.grid('--')
# ax.legend()

# # fig.savefig(f"../various plots/Total_efficiency_change_{this_batch}_{this_scope}_DUT{dut}_with_time_cut.svg" ,bbox_inches='tight')

## Now combine all efficiencies

In [ ]:
print(mins[this_scope][dut])

In [ ]:
# this_scope = "S2"
# # transimpedance = get_transimpedance(this_batch, this_scope) ### 4700 or  10700
# transimpedance = [dict_of_batches[this_batch].S[this_scope].get_sensor(f'ch{dut+1}').transimpedance for dut in (1,2,3)]

# dut = 2
# threshold_range = np.arange(-5, 20, 0.5)

# efficiency_pulse_cut = []
# error_pulse_cut = []

# efficiency_center = []
# error_center = []

# efficiency_time_cut = []
# error_time_cut = []

# efficiency_pulse_time_cut = []
# error_pulse_time_cut = []

# geometry = geometry_mask(df[this_scope], DUT_number=dut, bins=bins)[0]
# geometry_center = geometry_mask(df[this_scope], DUT_number=dut, bins=bins, only_select='center')[0]
# pulse_min = mins[this_scope][dut]
# pulseHeight = df[this_scope][f"pulseHeight_{dut}"]>pulse_min

# for charge in threshold_range:
#     ### efficiency with pulseHeight cut
#     eff, err = efficiency_error(df[this_scope][f"charge_{dut}"].loc[np.logical_and(geometry,pulseHeight)]/transimpedance[dut-1], charge)
#     efficiency_pulse_cut.append(eff)
#     error_pulse_cut.append(err) 
#     ### efficiency with geometry cut
#     eff, err = efficiency_error(df[this_scope][f"charge_{dut}"].loc[geometry_center]/transimpedance[dut-1], charge)
#     efficiency_center.append(eff)
#     error_center.append(err) 
#     ### efficiency with time cut
#     eff, err = efficiency_error(df[this_scope][f"charge_{dut}"].loc[np.logical_and(geometry,all_time_cut[dut-1])]/transimpedance[dut-1], charge)
#     efficiency_time_cut.append(eff)
#     error_time_cut.append(err)
#     ### efficiency with pulseHeight AND time cuts
#     eff, err = efficiency_error(df[this_scope][f"charge_{dut}"].loc[np.logical_and(geometry,np.logical_and(pulseHeight,all_time_cut[dut-1]))]/transimpedance[dut-1], charge)
#     efficiency_pulse_time_cut.append(eff)
#     error_pulse_time_cut.append(err)
    
    
# fig, ax = plt.subplots(figsize=(10,6), dpi=300)
# ### pulseHeight cut
# ax.errorbar(threshold_range, efficiency_pulse_cut, yerr=error_pulse_cut, marker='.', markersize=3, linewidth=0,
#             elinewidth=1, ecolor='k', capsize=1.5,
#             label=f"PulseHeight cut")#, #tracks: {df[this_scope][f'charge_{dut}'].loc[geometry].size}")
# ### central area
# ax.errorbar(threshold_range, efficiency_center, yerr=error_center, marker='.', markersize=3, linewidth=0,
#             elinewidth=1, ecolor='k', capsize=1.5,
#             label=f"Geometry cut")#, #tracks: {df[this_scope][f'charge_{dut}'].loc[geometry].size}")
# ### time cut
# ax.errorbar(threshold_range, efficiency_time_cut, yerr=error_time_cut, marker='.', markersize=3, linewidth=0,
#             elinewidth=1, ecolor='k', capsize=1.5,
#             label=f"Time cut")#, #tracks: {df[this_scope][f'charge_{dut}'].loc[geometry].size}")

# ### pulseHeight AND time cuts
# ax.errorbar(threshold_range, efficiency_pulse_time_cut, yerr=error_pulse_time_cut, marker='.', markersize=3, linewidth=0,
#             elinewidth=1, ecolor='k', capsize=1.5,
#             label=f"PulseHeight and Time cut")#, #tracks: {df[this_scope][f'charge_{dut}'].loc[geometry].size}")

# ax.set_title(f"Efficiencies depending on threshold charge \n \
# on full surface of {dict_of_batches[this_batch].S[this_scope].get_sensor(f'Ch{dut+1}'}, batch {this_batch}")
# ax.set_xlabel("Threshold charge (fC)",fontsize=16)
# ax.set_ylabel("Total efficiency",fontsize=16)
# ax.axhline(0.95, label="95% efficiency", color='r', alpha=0.4, linewidth=2)
# ax.grid('--')
# ax.legend(fontsize=16)
# # fig.savefig(f"../various plots/Total_efficiencies_change_{this_batch}_{this_scope}_DUT{dut}_different_cuts.svg")

### Efficiency for all sensors

In [ ]:
# this_scope = "S1"
# transimpedance = [dict_of_batches[this_batch].S[this_scope].get_sensor(f'ch{dut+1}').transimpedance for dut in DUTs]

# threshold_range = np.arange(-5, 20, 0.5)
# fig, axes = plt.subplots(figsize=(8*len(DUTs),6),ncols=len(DUTs), dpi=300)

# for i,dut in enumerate(DUTs):
#     efficiency_normal = []
#     error_normal = []
    
#     efficiency_center = []
#     error_center = []

#     geometry = geometry_mask(df[this_scope], DUT_number=dut, bins=bins)[0]
#     geometry_center = geometry_mask(df[this_scope], DUT_number=dut, bins=bins, only_select='center')[0]

#     for charge in threshold_range:
#         ### efficiency with geometry cut
#         eff, err = efficiency_error(df[this_scope][f"charge_{dut}"].loc[geometry]/transimpedance[dut-1], charge)
#         efficiency_normal.append(eff)
#         error_normal.append(err)
#         ### efficiency with geometry cut (center)
#         eff, err = efficiency_error(df[this_scope][f"charge_{dut}"].loc[geometry_center]/transimpedance[dut-1], charge)
#         efficiency_center.append(eff)
#         error_center.append(err) 
        
#     ### geometry cut
#     axes[i].errorbar(threshold_range, efficiency_normal, yerr=error_normal, marker='.', markersize=3, linewidth=0,
#                 elinewidth=1, ecolor='k', capsize=1.5,
#                 label=f"Full surface")#, #tracks: {df[this_scope][f'charge_{dut}'].loc[geometry].size}")
#     ### geometry cut (center)
#     axes[i].errorbar(threshold_range, efficiency_center, yerr=error_center, marker='.', markersize=3, linewidth=0,
#                 elinewidth=1, ecolor='k', capsize=1.5,
#                 label=f"Center")#, #tracks: {df[this_scope][f'charge_{dut}'].loc[geometry].size}")

#     axes[i].set_title(f" Sensor: {dict_of_batches[this_batch].S[this_scope].get_sensor(f'Ch{dut+1}').name}, batch {this_batch}",fontsize=16)
#     axes[i].set_xlabel("Threshold charge (fC)",fontsize=16)
#     axes[i].set_ylabel("Total efficiency",fontsize=16)
#     axes[i].axhline(0.95, label="95% efficiency", color='r', alpha=0.4, linewidth=2)
#     axes[i].axvline(4, label="4fC threshold", color='g', linewidth=1)
#     axes[i].grid('--')
#     axes[i].legend(fontsize=16)
# fig.suptitle(f"Efficiencies depending on threshold charge",fontsize=20,y=1.05)
# fig.savefig(os.path.join(pres_path,f"Efficiencies depending on threshold charge batch {this_batch} {this_scope}.png"), bbox_inches='tight')

In [ ]:
# # mins = {"S1":[0,53.8,52.1,65.3], "S2":[0,37.9,51.6,64.3]}
# # this_scope = "S2"
# this_DUT = 2 ### 1,2,3
# # transimpedance = get_transimpedance(this_batch, this_scope)[this_DUT-1]
# transimpedance = [dict_of_batches[this_batch].S[this_scope].get_sensor(f'ch{dut+1}').transimpedance for dut in (1,2,3)]


# pulseheight_cut = df[this_scope][f"pulseHeight_{this_DUT}"]>mins[this_scope][this_DUT]
# geometry_cut = geometry_mask(df[this_scope], bins=bins, bins_find_min='rice', DUT_number=this_DUT)[0]
# time_cut = all_time_cut[this_DUT-1]

# data_pulse_filter = df[this_scope][f"charge_{this_DUT}"].loc[pulseheight_cut]/transimpedance[this_DUT-1]
# # data_pulse_geo_filter = df[this_scope][f"charge_{this_DUT}"].loc[np.logical_and(pulseheight_cut, geometry_cut)]/transimpedance[this_DUT-1]
# data_geo_filter = df[this_scope][f"charge_{this_DUT}"].loc[geometry_cut]/transimpedance[this_DUT-1]
# data_time_filter = df[this_scope][f"charge_{this_DUT}"].loc[time_cut]/transimpedance[this_DUT-1]

# ### I need to figure out how to combine colors into nice thing
# fig, ax = plt.subplots(figsize=(10,6), dpi=300)
# hist, bins_for_all, _,_,_ = plot_histogram(df[this_scope][f"charge_{this_DUT}"]/transimpedance[this_DUT-1], bins='auto', label="no cut", fig_ax=(fig,ax), color=colormap[0], linewidth=1)
# plot_histogram(data_pulse_filter, bins=bins_for_all, fig_ax=(fig,ax), label="pulseheight cut", alpha=.5, color=colormap[1], histtype='stepfilled')
# # plot_histogram(data_pulse_geo_filter, bins=bins_for_all, fig_ax=(fig,ax), label="pulseheight & geom. cut", alpha=.5, color='#ff0000')
# plot_histogram(data_geo_filter, bins=bins_for_all, fig_ax=(fig,ax), label="geometry cut", alpha=.5, color=colormap[2], histtype='stepfilled') # #ffff00
# plot_histogram(data_time_filter, bins=bins_for_all, fig_ax=(fig,ax), label="time cut", alpha=.5, color=colormap[3], histtype='stepfilled') # #ffff00

# ax.set_xlabel("Charge [fC]", fontsize=20)
# ax.set_ylabel("Events (log)", fontsize=20)
# ax.set_title(f"Charge plot various cuts \nbatch:{this_batch}, {this_scope}, DUT:{this_DUT} ({dict_of_batches[this_batch].S[this_scope].get_sensor(f'Ch{dut+1}').name})",
#              fontsize=24,y=1.05)
# ax.grid('--')
# ax.set_xlim(-5,20)
# ax.semilogy()
# ax.legend(fontsize=16)

# fig.savefig(f"../various plots/charge_plot_various_cuts_{this_batch}_{this_scope}_DUT{this_DUT}.svg",bbox_inches='tight')

In [ ]:
import lecroyparser

In [ ]:
trc_file_path = "/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/Oscilloscope_raw_1/C1WF00000.trc"

data = lecroyparser.ScopeData(trc_file_path)

data.__dict__.keys()

In [ ]:
one_point=12000
plt.plot(data.x[one_point+0:one_point+1000], data.y[one_point+0:one_point+1000])